## Formants and Pitch
+ ###### Author: Dirk Van Compernolle   
+ ###### Modification History:   17/02/2023
+ ###### Requires:  pyspch>=0.7

### Description
In this notebook we want you to explore formants in pitch in speech data.
First you analyze prerecorded speech and later you can work with your own voice.



1. setting up:
    + work with Spd.iSpectrogram2()
    + load any speech waveform  (suggestions: misc/expansionist.wav, misc/friendly.wav)
    + use default spectrogram parameters   
    
    
2. Find pitch in time and/or frequency domain
    + put the range cursor in the middle of a vowel
    + find pitch in three ways: 
        - finding the period in the time waveform
        - find harmonic distance in a spectral slice (e.g. count how many in a 1kHz range)
        - find harmonic distance in a spectrogram
    + verify your measured pitch with pitch histograms in [Pitch vs. Gender Histograms](PitchGender.html)
        - was pitch a clear indication of gender for this person / voice segment ?
    + if you go to a different vowel, does the pitch change considerably or not ?
        
3. Find the formants in the spectrum
    + again, select a vowel
    + can you determin formant values from the spectrogram or spectral slice ?
        - remember to think in terms of spectral envelope !
        - the subtleties of the spectral envelope might be better visable in the spectral slice than on the spectrogram; do you agree ?
    + try to recognize the vowel on basis of first and second formant and then looking up in formant tables  [FormantTriangle](FormantTriangle.html)
    
    
3. Pitch and formants in the mel spectrum
    + for a readup on the mel-scale, check out [mel_scale.ipynb notebook](mel_scale.ipynb) in this folder use the plot in [mel_scale](mel_scale.png) as mapping reference
    + add the mel spectrogram (and mel spectrum slice) to your view
    + find the formants in the mel spectrum both for low resolution (nb=20-30) and high resolution (>80) mel filterbanks
    + in which representation is finding formants easiest ?
    + try to map formant frequencies to mel scale and filterbank channel (hint use a number of melbands equal to 30, 60 or 90 


illustrates various ways of audio recording in a Jupyter notebook
- either directly via command line
- either via the **iRecorder()** GUI 

#### iRecorder()

is a simple Jupyter Lab / ipywidgets based GUI for speech recordings.
- Record and Play Buttons
- set recording time and sampling frequency
- Save (selected part of) current recording to a named file

 

#### Writing Files on Colab

You can write into your temporarily allocated space on colab.
However, if you want to reuse files later, the way to go is to mount your google drive first and
write them there.
With the following lines of code, you will be able to access your personal google drive
> from google.colab import drive    
> drive.mount('/content/gdrive')    
> gdrive_path = '/content/gdrive/My Drive/'    

#### CAVEATS

A Jupyter notebook is not the optimal environment for interactive Audio I/O ,
due to a 3 way potential conflict between browser / latencies and rules in jupyter client-server architecture 
/ notebook.
Due to these various policy level, the notebook is never fully in control of what happens.
Moreover audio hardware capabilties vary drastically between machines.
For playback, resulting issues are manageable.  For recording, things are a bit more messy.

Issues that you may encounter include:
- lead time in recordings:  there may be a discrepancy between the prompt to start recording and the actual start of the recording; this can be substantial (a few 100 msec), just discard these unwanted parts when saving
- autoplay and autonormalize are features that can be overruled by the browser; you may see different behavior on Chrome vs. Firefox ..

In [1]:
# if needed, uncomment the pip install command to install pyspch -- it is required!
#
#!pip install git+https://github.com/compi1234/pyspch.git
#
try:
    import pyspch
except ModuleNotFoundError:
    try:
        print(
        """
        To enable this notebook on platforms as Google Colab, 
        install the pyspch package and dependencies by running following code:

        !pip install git+https://github.com/compi1234/pyspch.git
        """
        )
    except ModuleNotFoundError:
        raise

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

from IPython.display import display, clear_output, Audio, HTML
import ipywidgets as widgets
import numpy as np
import librosa

# verify the IPython version
import IPython
Audio_args = {'normalize':False}
if IPython.version_info[0] < 6:
    print("Warning: you are using IPython<6 which has limited Audio support\n")
    Audio_args = {}
import pyspch.core as Spch
import pyspch.sp as Sps
import pyspch.display as Spd
import pyspch.display.interactive as SpGui

## Setting up Google Drive, so that you can save files for future usage
You have two options for storing (recorded) files:
1. Locally inside your current session: just give a filename
2. On your Google Drive
    + first mount Google Drive for usage inside this notebook (as in the cell below)
    + A file in your Google Drive, e.g. 'MyFile.ext' is accessible in this notebook as:
        > 'gdrive/MyDrive/MyFile.ext'
        
By default we are setting up linkage with your Google Drive if on colab.  Don't execute if you don't need it.

In [4]:
try:
    from google.colab import drive    
    drive.mount('/content/gdrive') 
except:
    None

## 2. Record via the iRecorder() GUI
Normally you should start iRecorder() without any arguments.   
Only in case you want to modify the size of the spectrogram and/or GUI, you may want to use the parameters *figsize* and *dpi*.
In the GUI you have:
- Record Section:
    + Play, Record, Stop buttons
    + input for recording time and sampling frequency
- Save Section:
    + filename
    + start and end times of section to be saved

In [6]:
SpGui.iRecorder(sample_rate=8000)

iRecorder(children=(Output(layout=Layout(border_bottom='solid 1px black', border_left='solid 1px black', borde…

In [7]:
SpGui.iSpectrogram2(fname="misc/expansionist.wav")

iSpectrogram2(children=(VBox(children=(HBox(children=(Output(layout=Layout(width='66.0%')), Output(layout=Layo…

In [ ]:
try:
    a,b = Spch.audio.load("")
except:
    a = None

In [ ]:
a

In [9]:
help(Spch.audio.load)

Help on function load in module pyspch.core.audio:

load(resource, sample_rate=None, **kwargs)
    This is a tiny wrapper around librosa.load() to accomodate for specifying a resource
    both by url or filename.
    librosa.load() has the nice feature that it allows for sample rate conversion ( soundfile.read() does not )
    
    Parameters:
    -----------
        resource : string          
            url or file name
        sample_rate : int (optional)
            if given, resample to the target sampling rate
        **kwargs 
            extra parameters to be passed to librosa
            e.g. mono(boolean)
            
    Returns:
    --------
        wavdata : float-32 array, shape (n_samples, ) for mono or (n_channels, n_samples)
            the waveform data are scaled to [-1., 1.]
        sample_rate : int
            sampling rate of returned signal



In [19]:
data,sr = Spch.audio.load("https://homes.esat.kuleuven.be/~spchlab/data/misc/expansionist_16k.wav",sample_rate=8000)

In [20]:
help(Spch.audio.save)

Help on function save in module pyspch.core.audio:

save(filename, wavdata, sample_rate, **kwargs)
    Save a one or multi-d waveform data using soundfile



In [21]:
Spch.audio.save("expansionist.wav",data,sr)

In [18]:
!cwd

'cwd' is not recognized as an internal or external command,
operable program or batch file.


In [22]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.
